In [ ]:
import numpy as np
import pandas as pd
import requests
#import json
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import urllib.request
import pickle

In [ ]:
import nltk
import string
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords 
from nltk.corpus import wordnet

train_sub_df = pd.read_pickle(r'submissions_train_06032019.pkl')
test_sub_df = pd.read_pickle(r'submissions_test_06032019.pkl')

sub_df = pd.concat([train_sub_df,test_sub_df])
sub_df=sub_df.reset_index(drop=True)
sub_df = sub_df.values.tolist()

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)


def StopPunctTagLem(dataset):
    stop_words = set(stopwords.words('english'))
    exclude_punct = set(string.punctuation)
    add_set = {'...', '’', '``', '–', '”', "''", '‘', '“'}
    word_list = [nltk.word_tokenize(sub[6]) for sub in dataset]

    i = 0
    filtered_sub_all = []
    while i < len(word_list):
        filtered_sub = [w.lower() for w in word_list[i]]
        filtered_sub = [w.strip() for w in filtered_sub if not w in exclude_punct]
        filtered_sub = [w for w in filtered_sub if not w in stop_words]
        filtered_sub = [w for w in filtered_sub if not w in add_set]
        filtered_sub_all.append(filtered_sub)
        i += 1
        
    print('stop words and punctuation removed...')
        
    lemmatizer = WordNetLemmatizer()
    j = 0
    filtered_sub_lem = []
    while j < len(filtered_sub_all):
        sub = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in filtered_sub_all[j]]
        filtered_sub_lem.append(sub)
        j += 1
        if j % 1000 == 0:
            print(str(j)+'/'+str(len(dataset)))
    return filtered_sub_lem   

In [ ]:
#train_sub_preprocessed = StopPunctTagLem(train_sub_df)
#test_sub_preprocessed = StopPunctTagLem(test_sub_df)
sub_df_preprocessed = StopPunctTagLem(sub_df)

In [ ]:
#######Doc2vec#######
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

tagged_subs = [TaggedDocument(words=subs, tags=[str(i)]) for i, subs in enumerate(sub_df_preprocessed)]
#y_train = [sub[9] for sub in train_sub_df]
#y_test = [sub[9] for sub in test_sub_df]


In [ ]:
max_epochs = 50
vec_size = 50
alpha = 0.025

model_sub = Doc2Vec(vector_size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
               dm =1)
  
model_sub.build_vocab(tagged_subs)

In [ ]:
for iteration in range(max_epochs):
    print('iteration {0}'.format(iteration))
    model_sub.train(tagged_subs,
                total_examples=model_sub.corpus_count,
                epochs=iteration)
    # decrease the learning rate
    model_sub.alpha -= 0.0002
    # fix the learning rate, no decay
    model_sub.min_alpha = model_sub.alpha
    
model_sub.save('./d2v_sub.model')
print('Model saved as d2v_sub.model')
model=Doc2Vec.load('d2v_sub.model')

sub_all_doc2vec=[]
i = 0
while i < len(sub_all_df):
    sub_doc2vec = model.infer_vector(sub_all_df[i])
    sub_all_doc2vec.append(sub_doc2vec)
    i +=1
#print(sub_all_doc2vec)
#pd.DataFrame(sub_all_doc2vec)

In [ ]:
#####T-SNE#####
from sklearn.manifold import TSNE

X_tsne = TSNE(n_components=2, perplexity=40, verbose=2).fit_transform(sub_all_doc2vec)

#import matplotlib.pyplot as plt
#y_all = [sub[9] for sub in sub_df]
#


In [ ]:
#######IGNORE: plot of submission titles#######
plt.figure(figsize=(50, 25))
plt.subplot(121)
y_all = [sub[9] for sub in sub_df]
plt.scatter(X_tsne[:, 0], X_tsne[:, 1], c=y_all)

In [ ]:
scatter_sub= pd.DataFrame(X_tsne, columns = ['Dimension1', 'Dimension2'])
scatter_sub['y'] = y_all
scatter_sub['SubmissionTitle'] = [sub[6] for sub in sub_df]
scatter_sub.to_csv(r'submissions_plot_06032019.csv')